# Загрузка геологических данных 

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

from prob_geo_explore.config import RAW_DATA_DIR, PROCESSED_DATA_DIR

2026-02-24 15:32:03.152 | INFO     | prob_geo_explore.config:<module>:11 - PROJ_ROOT path is: /home/ab/apps/prob-geo-explore


### Загрузка блоков

In [3]:
pd.read_csv(PROCESSED_DATA_DIR / "block_ranked.csv")

,block_id,block_name_raw,region,block_area_km2,centroid_lon,centroid_lat,well_count,dry_well_count,water_depth_avg,water_depth_p50,...,near_field,dist_to_nearest_field_km,has_discovery_linked_to_field,commercial_success,seismic_good,seismic_poor,poorly_explored,many_dry_wells,deepwater_penalty,p_success
0,b15_11,15/11,North sea,0.958431,1.659586,58.243445,0,0,NaN,NaN,...,1,14.264258,False,0,0,0,1,0,0,0.042220
1,b24_11,24/11,North sea,13.144953,1.653965,59.040652,0,0,NaN,NaN,...,1,21.629947,False,0,0,0,1,0,0,0.042220
2,b24_3,24/3,North sea,0.833962,1.994076,59.756425,0,0,NaN,NaN,...,1,15.390853,False,0,0,0,1,0,0,0.042220
3,b25_12,25/12,North sea,531.657353,2.831699,59.124426,1,1,135.000000,135.0,...,1,19.482149,False,0,0,0,1,0,0,0.042220
4,b25_3,25/3,North sea,520.052919,2.831663,59.874454,1,1,114.000000,114.0,...,1,18.426670,False,0,0,0,1,0,0,0.042220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,b2_6,2/6,North sea,569.635364,3.831868,56.624351,25,4,61.940000,56.3,...,0,30.398042,False,0,0,0,0,1,0,0.008567
1823,b35_3,35/3,North sea,488.654267,3.831585,61.874544,7,3,262.428571,259.0,...,0,54.970436,False,0,0,0,0,1,0,0.008567
1824,b6204_10,6204/10,Norwegian sea,484.684412,4.164915,62.124558,4,3,176.000000,172.0,...,0,80.248231,False,0,0,0,0,1,0,0.008567
1825,b6306_6,6306/6,Norwegian sea,460.662946,6.831640,63.624653,3,3,240.433333,224.0,...,0,74.499659,False,0,0,0,0,1,0,0.008567


In [ ]:
blocks = gpd.read_file(RAW_DATA_DIR / "blkArea/blkArea.shp")
blocks = blocks.to_crs(epsg=4326)
blocks.head()

### Загрузка скважин

In [ ]:
wells = pd.read_csv(RAW_DATA_DIR / "wellbore_all_long.csv")
wells.head()

In [ ]:
geometry = [
    Point(xy) for xy in zip(wells["wlbEwDecDeg"], wells["wlbNsDecDeg"])
]
wells_gdf = gpd.GeoDataFrame(wells, geometry=geometry, crs="EPSG:4326")
wells_gdf.head()

### Пространственное присвоение скважин блокам

In [ ]:
well_blocks = gpd.sjoin(wells_gdf, blocks, how="left", predicate="within")
well_blocks.head()

### Агрегация признаков блока

In [ ]:
well_counts = well_blocks.groupby("block_name").size()
well_counts

In [ ]:
dry_wells = well_blocks[
    well_blocks["wlbContent"].str.contains("dry", case=False, na=False)
]
dry_counts = dry_wells.groupby("block_name").size()
dry_counts

In [ ]:
water_depth = well_blocks.groupby("block_name")["wlbWaterDepth"].mean()
water_depth

### Discoveries

In [ ]:
discoveries = pd.read_csv(RAW_DATA_DIR / "discovery.csv")
fields = pd.read_csv(RAW_DATA_DIR / "field.csv")

### Стратиграфия

In [ ]:
strat = pd.read_csv(RAW_DATA_DIR / "strat_litho_wellbore.csv")

### Near Field

In [ ]:
blocks["centroid"] = blocks.geometry.centroid